<a href="https://colab.research.google.com/github/ramrajv/NIFTY_500_Sentiment_Analysis/blob/main/playbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [2]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
nb_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0,nb_path)

Mounted at /content/drive


In [3]:
# !pip install --target=$nb_path nsepython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 74.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.4/499.4 KB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 KB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 KB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 KB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 KB 11.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that a

# Imports

In [4]:
import pandas as pd
import numpy as np
from urllib.request import urlopen, Request
import requests
from bs4 import BeautifulSoup
from pprint import pprint
from nsepython import *
import datetime
import pytz
#import time

import nltk
nltk.downloader.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import pandas as pd
import matplotlib.pyplot as plt
import plotly
import plotly.express as px

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


Get Company Sector, industry data

In [5]:
nifty_500_ticker_url = 'https://www1.nseindia.com/content/indices/ind_nifty500list.csv'
nifty_200_ticker_url = 'https://www1.nseindia.com/content/indices/ind_nifty200list.csv'
nifty_100_ticker_url = 'https://www1.nseindia.com/content/indices/ind_nifty100list.csv'
nifty_50_ticker_url = 'https://www1.nseindia.com/content/indices/ind_nifty50list.csv'
tickers_file = pd.read_csv(nifty_500_ticker_url)
tickers_df = tickers_file[['Symbol', 'Company Name']]
tickers = tickers_df['Symbol']

In [6]:
# Get articles and put them into a df
news_url = 'https://ticker.finology.in/company/'

In [8]:
# list to store article data
data = []
unavailable_tickers = []
companies_len = len(tickers)
length = companies_len
days_limit = datetime.datetime.now() - datetime.timedelta(days=30) #only 30 days old or newer articles
print('Fetching Article data..')
for i in range(length):
    print(i, tickers[i])
    url= '{}/{}'.format(news_url, tickers[i])
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0'}
    response = requests.get(url, headers=headers)
    html = BeautifulSoup(response.content, 'lxml')
    news_links = html.select('#newsarticles > a')
    if len(news_links) == 0:
        print('No news found for {}'.format(tickers[i]))
        unavailable_tickers.append(tickers[i])
        continue
    new_articles_counter = 0
    for link in news_links:
        title = link.find('span', class_='h6').text
        #separate date and time from datetime object
        date_time_obj = datetime.datetime.strptime(link.find('small').text, '%d %b %Y, %I:%M%p')
        if (date_time_obj <= days_limit):
             continue
        art_date = date_time_obj.date().strftime('%Y/%m/%d')
        art_time = date_time_obj.time().strftime('%H:%M')
        data.append([tickers[i], title, art_date, art_time])
        new_articles_counter += 1
    if(new_articles_counter==0):
        unavailable_tickers.append(tickers[i])  

df = pd.DataFrame(data, columns=['Ticker', 'Headline', 'Date', 'Time'])

print(unavailable_tickers)

Fetching Article data..
0 360ONE
1 3MINDIA
2 ABB
3 ACC
4 AIAENG
5 APLAPOLLO
6 AUBANK
7 AARTIDRUGS
8 AAVAS
9 ABBOTINDIA
10 ADANIENT
11 ADANIGREEN
12 ADANIPORTS
13 ATGL
14 ADANITRANS
15 AWL
16 ABCAPITAL
17 ABFRL
18 ABSLAMC
19 AEGISCHEM
20 AETHER
21 AFFLE
22 AJANTPHARM
23 APLLTD
24 ALKEM
25 ALKYLAMINE
26 ALOKINDS
27 AMARAJABAT
28 AMBER
29 AMBUJACEM
30 ANGELONE
31 ANURAS
32 APOLLOHOSP
33 APOLLOTYRE
34 APTUS
35 ASAHIINDIA
36 ASHOKLEY
37 ASIANPAINT
38 ASTERDM
39 ASTRAZEN
40 ASTRAL
41 ATUL
42 AUROPHARMA
43 AVANTIFEED
44 DMART
45 AXISBANK
46 BASF
47 BEML
48 BSE
49 BAJAJ-AUTO
50 BAJAJELEC
51 BAJFINANCE
52 BAJAJFINSV
53 BAJAJHLDNG
54 BALAMINES
55 BALKRISIND
56 BALRAMCHIN
57 BANDHANBNK
58 BANKBARODA
59 BANKINDIA
60 MAHABANK
61 BATAINDIA
62 BAYERCROP
63 BERGEPAINT
64 BDL
65 BEL
66 BHARATFORG
67 BHEL
68 BPCL
69 BHARATRAS
70 BHARTIARTL
71 BIOCON
72 BIRLACORPN
73 BSOFT
74 BLUEDART
75 BLUESTARCO
76 BBTC
77 BORORENEW
78 BOSCHLTD
79 BRIGADE
80 BCG
81 BRITANNIA
82 MAPMYINDIA
83 CCL
84 CESC
85 CGPOWER
86 

## removing unavailable tickers from original df

In [9]:
tickers = np.setdiff1d(tickers, unavailable_tickers)
tickers_df.drop(tickers_df[tickers_df['Symbol'].isin(unavailable_tickers)].index.values, inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


# Sentiment Analysis

In [10]:
print('Performing Sentiment Analysis')
vader = SentimentIntensityAnalyzer()
scores = df['Headline'].apply(vader.polarity_scores).tolist()
scores_df = pd.DataFrame(scores)
new_df = pd.merge(left=df, right=scores_df, on=df.index.values).drop(['key_0'], axis=1)
final_df = new_df.groupby('Ticker').mean()

Performing Sentiment Analysis


# Get Company,Sector,Industry,mCap Data

In [11]:
tickers[0]

'360ONE'

In [12]:
len(tickers)

378

In [15]:
# sector = []
# industry = []
mCap = []
companyName = []
new_length = len(tickers)
#new_length = 10
print('Fetching industry data')
for i in range(new_length):
    meta = nse_eq(tickers[i])
#     print(i, tickers[i])
#     try:
#         sector.append(meta['industryInfo']['macro'])
#     except KeyError:
#         print('{} is not available'.format(tickers[i]))
#         sector.append(np.nan)
#         industry.append(np.nan)
#         mCap.append(np.nan)
#         companyName.append(np.nan)
#         continue
#     #pprint('Sector: {}'.format(meta['industryInfo']['macro']))
#     try:
#         industry.append(meta['industryInfo']['sector'])
#     except KeyError:
#         print('{} is not available'.format(tickers[i]))
#         industry.append(np.nan)
#         mCap.append(np.nan)
#         companyName.append(np.nan)
#         continue
    # pprint('Industry: {}'.format(meta['industryInfo']['sector']))
    try:
        mCap.append(round((meta['priceInfo']['previousClose'] * meta['securityInfo']['issuedSize'])/1000000000, 2))
    except KeyError:
        print('{} is not available'.format(tickers[i]))
        mCap.append(np.nan)
        companyName.append(np.nan)
        continue
    try:
        companyName.append(meta['info']['companyName'])
    except KeyError:
        print('{} is not available'.format(tickers[i]))
        companyName.append(np.nan)
    
    #print('market cap is Rs {}'.format(ticker_mcap))
    #print('\n')

final_df['mCap (Billion)'] = mCap
final_df['Company Name'] = companyName

Fetching industry data


In [16]:
final_df = final_df.reset_index()
# final_df = pd.merge(final_df, xc_indices, left_on='Ticker', right_on='Ticker', how='inner')
final_df.columns = ['Symbol', 'Negative', 'Neutral', 'Positive', 'Sentiment Score','MCap (Billion)', 'Company Name']

In [17]:
final_df = final_df.dropna()
final_df = final_df.round(3)

final_df.to_csv('sentiment_data.csv')

In [18]:
# Plotting
print('Generating Plots')
fig = px.treemap(
    final_df, path=[px.Constant('Nifty 500'), 'Symbol'], color='Sentiment Score',
    hover_data=['Company Name', 'Negative', 'Neutral', 'Positive', 'Sentiment Score'], color_continuous_scale=['#FF0000', "#000000", '#00FF00'], color_continuous_midpoint=0
    )
#values='MCap (Billion)', 
fig.data[0].customdata = final_df[['Company Name', 'Negative', 'Neutral', 'Positive', 'Sentiment Score']]
fig.data[0].texttemplate = "%{label}<br>%{customdata[4]}"
fig.update_traces(textposition="middle center")
fig.update_layout(margin = dict(t=30, l=10, r=10, b=10), font_size=20)

Generating Plots


In [19]:
sentiscore = final_df[['Company Name','Sentiment Score']]
sentiscore = sentiscore.sort_values('Sentiment Score', ascending = False)

In [20]:
sentiscore.to_csv('SentimentalScore.csv')

In [21]:
# import kaleido
fig.write_html('image.html')

In [22]:
# Get current date, time and timezone to print to the html page
# now = datetime.datetime.now()
# ist_timezone = pytz.timezone('Asia/Kolkata')
# dt_string = now.astimezone(ist_timezone).strftime("%d/%m/%Y %H:%M:%S")

In [23]:
# Generate HTML File with Updated Time and Treemap
# print('Writing HTML')
# with open('NIFTY_500_live_sentiment.html', 'a') as f:
#     f.truncate(0) # clear file if something is already written on it
#     title = "<h1>NIFTY 500 Stock Sentiment Dashboard</h1>"
#     updated = "<h2>Last updated: " + dt_string + " (Timezone: IST" + ")</h2>"
#     description = "This dashboard is updated at 17:30 IST Every Day with sentiment analysis performed on latest scraped news headlines.<br><br>"
#     f.write(title + updated + description)
#     f.write(fig.to_html(full_html=False, include_plotlyjs='cdn')) # write the fig created above into the html file

In [24]:
sentiscore.columns

Index(['Company Name', 'Sentiment Score'], dtype='object')

In [33]:
tnow = datetime.date.today()

In [34]:
tnow

datetime.date(2023, 1, 25)

In [36]:
sentiscore.rename(columns={'Sentiment Score':'SSCore'+' '+str(tnow)})

,Company Name,SSCore 2023-01-25
40,Balrampur Chini Mills Limited,0.832
201,Jubilant Pharmova Limited,0.765
143,Happiest Minds Technologies Limited,0.714
33,Adani Wilmar Limited,0.710
119,Firstsource Solutions Limited,0.663
...,...,...
361,UFLEX Limited,-0.318
32,Avanti Feeds Limited,-0.318
245,Mangalore Refinery and Petrochemicals Limited,-0.318
351,Torrent Power Limited,-0.318
